In [15]:
# data analysis and preprocessing 
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
import re

# Model evaluation modules
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib


# ML models
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier


### News Data overview

In [3]:
news=pd.read_csv('train.csv')
print(news.shape)
news.head(8)

(20800, 5)


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0


#### Most active Authors with more than 100 news articles

In [4]:
auth=news['author'].value_counts()
auth=auth[auth>100]
auth

Pam Key                243
admin                  193
Jerome Hudson          166
Charlie Spiering       141
John Hayward           140
Katherine Rodriguez    124
Warner Todd Huston     122
Ian Hanchett           119
Breitbart News         118
Daniel Nussbaum        112
Jeff Poor              107
AWR Hawkins            107
Joel B. Pollak         106
Trent Baker            102
Name: author, dtype: int64

#### Most active authors with coresponding number of fake/authentic news articles 
#### (0-fake, 1-authentic)

In [5]:
auth_count=news.groupby(['author','label'])['label'].count()

auth=list(auth.index)    # get list of top active authors with over 100 news articles
auth_count[auth]

author               label
AWR Hawkins          0        107
Breitbart News       0        118
Charlie Spiering     0        141
Daniel Nussbaum      0        112
Ian Hanchett         0        119
Jeff Poor            0        107
Jerome Hudson        0        166
Joel B. Pollak       0        106
John Hayward         0        140
Katherine Rodriguez  0        124
Pam Key              0        242
                     1          1
Trent Baker          0        102
Warner Todd Huston   0        122
admin                1        193
Name: label, dtype: int64

#### From the above output, except  author 'Admin', all other most active authors post 'fake news'

### Remove missing values and reset index

In [6]:
news=news.dropna()
news.reset_index(inplace=True)
news.shape

(18285, 6)

### Data preprocessing: Stemming, removing stopwords and non-alphanumeric characters

In [7]:
ps=PorterStemmer()
tit=[]
for n in range(0,len(news)):
    msg=re.sub('[^a-zA-Z0-9]',' ',news['title'][n])
    msg=msg.lower()
    msg=word_tokenize(msg)          # split text message into a list
    msg=[ps.stem(w) for w in msg if not w in stopwords.words('english')]
    msg = ' '.join(msg)    # Join list back into sentence
    tit.append(msg)



In [8]:
tit[0:5]  # First 5 titles

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 '15 civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

### Split news dataset into training and testing data

In [9]:
# Split dataset into 80% for training and 20% testing 
from sklearn.model_selection import train_test_split 

Y=news.iloc[:,-1]
x_train, x_test, y_train, y_test = train_test_split(tit,Y, test_size=0.2, random_state=50)


In [10]:
print('length of x_train: %d and y_train:%d ' %(len(x_train),len(y_train)))

length of x_train: 14628 and y_train:14628 


### Constructing Pipelines

In [11]:
pipe_lr = Pipeline([('vect', CountVectorizer(max_features=5000)),
                   ('tfidf', TfidfTransformer()),
                ('classifier', LogisticRegression(random_state=42))])

pipe_rf = Pipeline([('vect', CountVectorizer(max_features=5000)),
                   ('tfidf', TfidfTransformer()),
            ('classifier', RandomForestClassifier(random_state=42))])

pipe_svm = Pipeline([('vect', CountVectorizer(max_features=5000)),
                   ('tfidf', TfidfTransformer()),
            ('classifier', svm.SVC(random_state=42))]) 

### Grid search parameters

In [12]:

param_range = [9, 10]
param_range_fl = [1.0, 0.5]

grid_params_lr = [{'classifier__penalty': ['l1', 'l2'],
        'classifier__C': param_range_fl,
        'classifier__solver': ['liblinear']}] 


grid_params_rf = [{'classifier__criterion': ['gini', 'entropy'],
        'classifier__max_depth': param_range,
        'classifier__min_samples_split': param_range[1:]}]

grid_params_svm = [{'classifier__kernel': ['linear', 'rbf'], 
        'classifier__C': param_range}]


### Grid searches

In [13]:
jobs = -1

LR = GridSearchCV(estimator=pipe_lr,
            param_grid=grid_params_lr,
            scoring='accuracy',
            cv=10) 



RF = GridSearchCV(estimator=pipe_rf,
            param_grid=grid_params_rf,
            scoring='accuracy',
            cv=10, 
            n_jobs=jobs)


SVM = GridSearchCV(estimator=pipe_svm,
            param_grid=grid_params_svm,
            scoring='accuracy',
            cv=10,
            n_jobs=jobs)

### Finding best classification model

In [14]:

grd = [SVM,RF,LR]

grd_dict = {0: 'Support Vector Machine', 
        1: 'Random Forest',
        2: 'Logistic Regression'}

# Fit the grid search objects
print('Model and parameter optimizations...')
best_acc = 0.0
best_clf = 0
best_gs = ''
for idx, gs in enumerate(grd):
    print('\nEstimator: %s' % grd_dict[idx])
    gs.fit(x_train, y_train)
    print('Best params are : %s' % gs.best_params_)
    # Best training data accuracy
    print('Best training accuracy: %.3f' % gs.best_score_)
    # Predict on test data with best params
    y_pred = gs.predict(x_test)
    # Test data accuracy of model with best params
    print('Test set accuracy score for best params: %.3f ' % accuracy_score(y_test, y_pred))
    # Track best (highest test accuracy) model
    if accuracy_score(y_test, y_pred) > best_acc:
        best_acc = accuracy_score(y_test, y_pred)
        best_gs = gs
        best_clf = idx
print('\nClassifier with best test set accuracy: %s' % grd_dict[best_clf])

Model and parameter optimizations...

Estimator: Support Vector Machine
Best params are : {'classifier__C': 9, 'classifier__kernel': 'rbf'}
Best training accuracy: 0.934
Test set accuracy score for best params: 0.933 

Estimator: Random Forest
Best params are : {'classifier__criterion': 'gini', 'classifier__max_depth': 10, 'classifier__min_samples_split': 10}
Best training accuracy: 0.888
Test set accuracy score for best params: 0.897 

Estimator: Logistic Regression
Best params are : {'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear'}
Best training accuracy: 0.923
Test set accuracy score for best params: 0.924 

Classifier with best test set accuracy: Support Vector Machine
